<a href="https://colab.research.google.com/github/hardik-sh/Stock-Prediction/blob/main/Stock_Prediction_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
import pandas_datareader.data as web
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
plt.style.use('bmh')
import yfinance as yfin
yfin.pdr_override()
df = web.DataReader('SBIN.NS',data_source="google",start='2012-01-01',end='2020-01-01')
df

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(16,8))
plt.title('SBI')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.plot(df['Close'])
plt.show()


In [ ]:
from sklearn.preprocessing import MinMaxScaler


In [ ]:
df=df.filter(['Close'])
df =df.dropna()
dataset = df.values
df.head(4)

In [ ]:
f_d = 250
df['Prediction']=df['Close'].shift(-f_d)
df.head(4)

In [ ]:
X = np.array(df.drop(['Prediction'], 1))[:-f_d]
print(X)

In [ ]:
y = np.array(df['Prediction'])[:-f_d]
print(y)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data =scaler.fit_transform(dataset)
scaled_data

In [ ]:
training_data_len =math.ceil(len(dataset)*.8)
training_data_len


In [ ]:
train_data = scaled_data[0:training_data_len , :]
x_train =[]
y_train =[]
for i in range(250,len(train_data)):
  x_train.append(train_data[i-250:i,0])
  y_train.append(train_data[i,0])
  if i<= 251:
    print(x_train)
    print(y_train)
    print()

In [ ]:
x_train,y_train =np.array(x_train),np.array(y_train)
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense, Activation 
from keras.layers import LSTM

In [ ]:
model =Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')


In [ ]:
model.fit(x_train, y_train, batch_size=10,epochs=4)


In [ ]:
test_data = scaled_data[training_data_len -250 :,:]
x_test = []
y_test = dataset[training_data_len:,:]
for i in range(250, len(test_data)):
  x_test.append(test_data[i-250:i,0])


In [ ]:
x_test = np.array(x_test)


In [ ]:
x_test = np.reshape(x_test,(x_test.shape[0], x_test.shape[1],1))


In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse


In [ ]:
train = df[:training_data_len]
valid =  df[training_data_len:]
valid['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date',fontsize =18)
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','Val','Predictions'], loc='lower right')
plt.show()


In [ ]:
valid


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)


In [ ]:
tree = DecisionTreeRegressor().fit(x_train,y_train)
lr= LinearRegression().fit(x_train,y_train)


In [ ]:
x_train.shape


In [ ]:
x_f = df.drop(['Prediction'],1)[:-f_d]
x_f=x_f.tail(f_d)
x_f=np.array(x_f)
x_f


In [ ]:
tree_p =tree.predict(x_f)
print(tree_p)
print()
lr_p=lr.predict(x_f)
print(lr_p)


In [ ]:
pr =tree_p
valid = df[X.shape[0]:]
valid['Predictions']=pr
plt.figure(figsize=(16,8))
plt.xlabel('Days')
plt.ylabel('Close Price')
plt.plot(df['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Orig','Val','Pred'])
plt.show()


In [ ]:
valid

In [ ]:
pr =lr_p
valid = df[X.shape[0]:]
valid['Predictions']=pr
plt.figure(figsize=(16,8))
plt.xlabel('Days')
plt.ylabel('Close Price')
plt.plot(df['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Orig','Val','Pred'])
plt.show()
